<a href="https://colab.research.google.com/github/perfectism13/learning_colab/blob/master/MDAD/rgb/M5_dataset_MDAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir(r'/content/drive/My Drive/MDAD/RGB1/')
# os.chdir(r'/content/drive/My Drive/imgs/')
# os.chdir(r'/content/drive/My Drive/data/imgs/')
print(os.getcwd())
!ls
!pip install keras==2.1.5
import keras
print(keras.__version__)
!nvidia-smi

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/MDAD/RGB1
driver_id_class_jpg.csv		       S10  S17  S23  S3   S36	S42  S49
driver_id_class_jpg_fast.csv	       S11  S18  S24  S30  S37	S43  S5
driver_id_class_jpg.gsheet	       S12  S19  S25  S31  S38	S44  S50
my_model12_MDAD_Nadam_batchsize_32.h5  S13  S2	 S26  S32  S39	S45  S6
my_model1_MDAD_batchsize_32.h5	       S14  S20  S27  S33  S4	S46  S7
my_model5_MDAD_batchsize_32.h5	       S15  S21  S28  S34  S40	S47  S8
S1				       S16  S22

Using TensorFlow backend.


2.1.5
Mon Feb 17 01:43:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
| 

In [0]:
import os
os.environ['KERAS_BACKEND']='tensorflow'
import keras

In [0]:
import numpy as np
np.random.seed(2017)

import os
import glob
import cv2
import math
import pickle
import pandas as pd
import random
from tqdm import tqdm
from matplotlib import pyplot as plt

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import SGD, Adam, Nadam
from keras.utils import np_utils
from keras.models import model_from_json
from sklearn.metrics import log_loss

### 图片读取函数
针对Xception模型进行图片读取和预处理。具体而言，使用OpenCV库从路径获取图片，尺寸改变为299*299*3。通道转换为RGB顺序。像素归一化。

In [0]:
# Load a preprocessed image
def get_im_cv2(path, img_size):
    img = cv2.imread(path)
    img = np.array(img, dtype=np.float32)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # reduce size
    img = cv2.resize(img, (img_size, img_size))
    # normalization
    img /= 127.5
    img -= 1.
    #print (img[1:5, 1:5, 0])
    return img

### 图片读取函数，附加实时增强
针对Xception模型进行图片读取和预处理。具体而言，使用OpenCV库从路径获取图片。然后进行图片增强处理，绕图片中心点随机旋转-10到10度，并在横向上平移-50到50个pixel。这个范围是观察一些图片后，根据不同图片之间拍摄的角度和位置差异，决定在这个范围内做随机增强。目的是加强模型的泛化能力，防止过拟合。
和上一个函数一样，也针对Xception模型进行预处理。尺寸改变为299*299*3。通道转换为RGB顺序。像素归一化。

In [0]:
from keras.preprocessing import image

# image rotation
def rotate(x, degree, row_axis=0, col_axis=1, channel_axis=2, fill_mode='wrap', cval=0.):
    theta = np.pi / 180 * degree
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta), 0],
    [np.sin(theta), np.cos(theta), 0],
    [0, 0, 1]])
    h, w = x.shape[row_axis], x.shape[col_axis]
    transform_matrix = image.transform_matrix_offset_center(rotation_matrix, h, w)
    x = image.apply_transform(x, transform_matrix, channel_axis, fill_mode, cval)
    return x

# image shift
def shift(x, wshift, hshift, row_axis=0, col_axis=1, channel_axis=2, fill_mode='wrap', cval=0.):
    h, w = x.shape[row_axis], x.shape[col_axis] #读取图片的高和宽
    tx = hshift * h #高偏移大小，若不偏移可设为0，若向上偏移设为正数
    ty = wshift * w #宽偏移大小，若不偏移可设为0，若向左偏移设为正数
    translation_matrix = np.array([[1, 0, tx],
                                  [0, 1, ty],
                                  [0, 0, 1]])
    transform_matrix = translation_matrix  
    x = image.apply_transform(x, transform_matrix, channel_axis, fill_mode, cval)
    return x

# PCA
def RGB_PCA(images):
    pixels = images.reshape(-1, images.shape[-1])
    m = np.mean(pixels, axis=0)
    pixels -= m
    C = np.cov(pixels, rowvar=False)
    l, v = np.linalg.eig(C)
    idx = np.argsort(l)[::-1]
    v = v[:,idx]
    l = l[idx]
    #print (C.shape, len(l), len(v))
    return l, v
def RGB_variations(image, eig_val, eig_vec):
    a = 0.1 * np.random.randn(3)
    v = np.array([a[0]*eig_val[0], a[1]*eig_val[1], a[2]*eig_val[2]])
    variation = np.dot(eig_vec, v)
    return image + variation

# change HSV
def randomHueSaturationValue(image, hue_shift_limit=(-10, 10),
                            sat_shift_limit=(-75, 75),
                            val_shift_limit=(-75, 75), u=0.5):
    if np.random.random() < u:
        img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        h, s ,v = img[:,:,0],img[:,:,1],img[:,:,2]
        hue_shift = np.random.uniform(hue_shift_limit[0], hue_shift_limit[1])
        h = h + hue_shift

        sat_shift = np.random.uniform(sat_shift_limit[0], sat_shift_limit[1])
        s = s + sat_shift

        val_shift = np.random.uniform(val_shift_limit[0], val_shift_limit[1])
        v = v + val_shift

        img[:,:,0],img[:,:,1],img[:,:,2] = h, s ,v
        image = cv2.cvtColor(img, cv2.COLOR_HSV2BGR)

    return image


In [0]:
# Load a random augmented image
def get_im_cv2_aug(path, img_size):
    img = cv2.imread(path)
    img = np.array(img, dtype=np.float32)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    #random rotate
    
    deg = random.uniform(-10, 10) #random rotate limit
    img = rotate(img, deg)
    
    #random shift
    wshift = random.uniform(-0.1, 0.1)
    hshift = random.uniform(-0.1, 0.1)
    img = shift(img, wshift, hshift)
    
    # change HSV
    #img = randomHueSaturationValue(img)
    
    # PCA
    #img = img/255.0
    #l, v = RGB_PCA(img)
    #img = RGB_variations(img, l, v)
    #img = img * 255.0
    
    # reduce size
    img = cv2.resize(img, (img_size, img_size))
    
    # normalization
    
    img /= 127.5
    img -= 1.

    return img

### 切分数据
按照司机的id来切分训练数据集。把driver_imgs_list分成两个列表。一个是训练列表，里面是所有训练集司机的图片文件名。另一个是验证列表，里面是所有验证集司机的图片文件名。然后把两个列表都随机打乱。 这里列表里面保存的都只是excel文件里的一行行数据（包括文件名，分类，司机id），而不是图片本身。等训练时通过生成器读取图片，这样节约内存。 另外还保存了一个类别字典，便于以后从同一类中抽取图片进行拼接。

In [0]:
# divide drivers
unique_list_train = []
unique_list_valid = []
for i in range(12):
  unique_list_train.append('S'+str(i+35))
print(unique_list_train)
for i in range(3):
  unique_list_valid.append('S'+str(i+47))
print(unique_list_valid)
# unique_list_train = ['p002', 'p012', 'p014', 'p015', 'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049', 'p050', 'p051', 'p052', 'p056', 'p061', 'p064', 'p066', 'p072', 'p075']
# unique_list_valid = ['p016', 'p021', 'p022', 'p024']
#print (unique_list_train, unique_list_valid)

# get index: driver_id, class, image name
index = os.path.join('.', 'driver_id_class_jpg.csv')

# build the driver id dictionary and class dictionary
f = open(index, 'r')
id_dict = dict()
class_dict = dict()
lines = f.readlines()
# print(lines[0])
# for line in lines[1:]:
#     arr = line.strip().split(',')
#     if arr[0] not in id_dict.keys():
#         id_dict[arr[0]] = [line]
#     else:
#         id_dict[arr[0]].append(line)
#     if arr[1] not in class_dict.keys():
#         class_dict[arr[1]] = [line]
#     else:
#         class_dict[arr[1]].append(line)
# f.close()

# split the train list and valid list by id
train_list = []
valid_list = []
# for id in id_dict.keys():
#     if id in unique_list_train:
#         train_list.extend(id_dict[id])
#     elif id in unique_list_valid:
#         valid_list.extend(id_dict[id])
for line in lines[1:]:
    arr = line.strip().split(',')
    if arr[0]  in unique_list_train:
        train_list.append(line)
    elif arr[0]  in unique_list_valid:
      valid_list.append(line)
f.close()
random.shuffle(train_list)
random.shuffle(valid_list)

print (len(train_list), len(valid_list))

['S35', 'S36', 'S37', 'S38', 'S39', 'S40', 'S41', 'S42', 'S43', 'S44', 'S45', 'S46']
['S47', 'S48', 'S49']
28547 8886


### 转换为One Hot Encode标签

对分类标签进行One Hot Encode的函数如下。

In [0]:
# one hot encode the class label

from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
class_id = []
for i in range(16):
  class_id.append('AC'+str(i+1))
lb.fit(class_id)   
def one_hot_encode(x):
    return lb.transform(x)
t = one_hot_encode(class_id)
print(t)

[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]]


### 训练图片生成器

从训练列表中遍历。yield一个batch的训练图片及其标签。图片经过了实时增强。另外还有50%的概率随机选取另一张同类里的图片，将两张的左右各半边拼接在一起。这也是为了训练模型对分类的关键部位进行学习，而不是记住司机的样子

In [0]:
# define my train data generator
def train_gen(batch_size):
    #random.shuffle(train_list) # 每一代都随机shuffle训练集
    current = 0
    while 1:
        x = []
        y = []
        while len(y) < batch_size:
            line = train_list[current]
            arr = line.strip().split(',')
            path1 = os.path.join('.', str(arr[0]), str(arr[1]),str(arr[2]))
            img = get_im_cv2_aug(path1, 299)
            # if random.random()>0.5:
            #     line2 = random.choice(class_dict[arr[1]])
            #     bname = line2.strip().split(',')[2]
            #     path2 = os.path.join('.', 'train', str(arr[1]), str(bname))
            #     img2 = get_im_cv2_aug(path2, 299)
            #     left = img[:, :150, :]
            #     right = img2[:, 150:, :]
            #     img = np.concatenate((left, right), axis=1)
            x.append(img)
            label = one_hot_encode([str(arr[1])])[0]
            y.append(label)
            current += 1
            if current >= len(train_list):
                current = 0
        x = np.array(x)
        x = x.reshape(batch_size, img_size, img_size, 3) 
        y = np.array(y, dtype = np.uint8)
        y = y.reshape(batch_size, 16)

        yield (x, y)

### 验证图片生成器

从验证列表中遍历。yield一个batch的验证图片及其标签。为了体现模型训练后的拟合能力，图片没有进行实时增强。

In [0]:
# define my validation data generator

def valid_gen(batch_size):
    current = 0
    while 1:
        x = []
        y = []
        while len(y) < batch_size:
            line = valid_list[current]
            arr = line.strip().split(',')
            path = os.path.join('.', str(arr[0]), str(arr[1]),str(arr[2]))
            #print (path)
            img = get_im_cv2(path, 299)
            x.append(img)
            label = one_hot_encode([str(arr[1])])[0]
            y.append(label)
            current += 1
            if current >= len(valid_list):
                current = 0
        x = np.array(x)
        x = x.reshape(batch_size, img_size, img_size, 3) 
        y = np.array(y, dtype = np.uint8)
        y = y.reshape(batch_size, 16)
        yield (x, y)

### 构建模型

用keras构建模型。使用在ImageNet上预训练好的Xception模型，接上一个global average pooling层，dropout防止过拟合，最后一个全连接层输出10个类别的概率。在全连接层的权重上采用了L2正则化。锁定模型的前70层不更新权重.

In [0]:
from keras import backend as K
K.clear_session()

In [0]:
# new model

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

img_size = 299
x = Input((img_size, img_size, 3))
base_model = InceptionV3(input_tensor = x, weights='imagenet', include_top=False)


x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(16, activation='softmax', kernel_regularizer=regularizers.l2(0.01))(x)
model = Model(base_model.input, x)

for i in range(95):
    model.layers[i].trainable = False
    
model.summary()
len(model.layers)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, 149, 149, 32) 96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_95 (Activation)      (None, 149, 149, 32) 0           batch_normalization_95[0][0]     
__________________________________________________________________________________________________
conv2d_96 

314

### 训练
进行模型训练。这里batch size为64。用了自适应优化器Nadam，使用schedule learning rate自动调整学习率的方法并在验证loss不下降时early stopping。一共训练5代，最后的验证集loss仍然有下降空间。steps per epoch设定为在一个epoch内所有训练图片被遍历1次.

In [0]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
model.compile(optimizer=Nadam(),loss='categorical_crossentropy',metrics=['categorical_accuracy'])

def learning_rate(epoch):
    ini_lr = 0.002
    lr = ini_lr * pow(10, -epoch)
    return lr

cp = ModelCheckpoint(filepath="my_model5_MDAD_batchsize_32.h5", save_best_only=True)
es = EarlyStopping()
lrs = LearningRateScheduler(learning_rate)
hist = model.fit_generator(train_gen(32), steps_per_epoch = 893, epochs = 5, workers=4, max_q_size=20, use_multiprocessing=True, validation_data = valid_gen(32), validation_steps = 278, callbacks=[cp, es, lrs])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=893, epochs=5, workers=4, use_multiprocessing=True, validation_data=<generator..., validation_steps=278, callbacks=[<keras.ca..., max_queue_size=20)`
  if sys.path[0] == '':


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:2087: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/5
892/893 [============================>.] - ETA: 6s - loss: 0.6085 - categorical_accuracy: 0.8797 

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:2330: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


893/893 [==============================] - 6426s 7s/step - loss: 0.6087 - categorical_accuracy: 0.8796 - val_loss: 3.3798 - val_categorical_accuracy: 0.3449
Epoch 2/5
879/893 [============================>.] - ETA: 1:18 - loss: 0.2621 - categorical_accuracy: 0.9527